# Advanced Data and Indicator Validation

This notebook provides a professional-grade workflow for validating historical data and technical indicator calculations. It uses reusable utility functions to handle instrument-specific data cleaning and visualization.

### The Validation Process:

1.  **Fetch Data:** We download historical data for a specific instrument.
2.  **Get Instrument Type:** We fetch the instrument's details to determine its type (e.g., Crypto or Share).
3.  **Clean Data:** We use our smart `clean_trading_data` utility, which applies the correct cleaning logic based on the instrument type.
4.  **Calculate Indicator:** We calculate the RSI on the cleaned dataset.
5.  **Interactive Visualization:** We use `Plotly` and our `create_rangebreaks_for_epic` utility to create an accurate, interactive chart for precise comparison with a source like TradingView.

### 1. Setup and Imports

In [ ]:
import os
import sys
from pathlib import Path

import pandas as pd
import plotly.graph_objects as go
import talib
from loguru import logger
from plotly.subplots import make_subplots

# Add the project root to the Python path
notebook_dir = Path(os.getcwd())
project_root = notebook_dir.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.capitalcom_bot.client_factory import get_client  # noqa: E402

# Import our new utility functions
from src.capitalcom_bot.utils.data_processing import clean_trading_data  # noqa: E402
from src.capitalcom_bot.utils.plotting import create_rangebreaks_for_epic  # noqa: E402

logger.remove()
logger.add(sys.stderr, level="INFO")

### 2. Configuration and Data Fetching

In [ ]:
# --- CONFIGURATION ---
# Try 'ETHUSD' (crypto) and 'TSLA' (stock) to see the different handling
EPIC_TO_VALIDATE = "BOTZ"
RESOLUTION = "HOUR"
DATA_POINTS = 500
RSI_PERIOD = 14

# --- DATA FETCHING ---
df_raw = pd.DataFrame()
instrument_type = ""

try:
    with get_client(demo_mode=True) as client:
        logger.info(f"Fetching data and details for {EPIC_TO_VALIDATE}...")
        # Fetch both historical prices and instrument details in one session
        df_raw = client.get_historical_prices(EPIC_TO_VALIDATE, RESOLUTION, DATA_POINTS)
        details = client.get_full_market_details(EPIC_TO_VALIDATE)
        instrument_type = details.instrument.type
        logger.success(
            f"Successfully fetched {len(df_raw)} data points for {instrument_type} instrument."
        )
except Exception as e:
    logger.error(f"Failed to fetch data: {e}", exc_info=True)

### 3. Data Cleaning and Indicator Calculation

Here we use our reusable `clean_trading_data` function. It will automatically apply the correct logic (keep or remove weekends) based on the `instrument_type` we fetched in the previous step.

In [ ]:
if not df_raw.empty:
    # Use the smart cleaning function
    df_cleaned = clean_trading_data(df_raw, instrument_type)

    # Calculate indicator on the cleaned data
    df_cleaned["rsi"] = talib.RSI(df_cleaned["close"], timeperiod=RSI_PERIOD)

    print("\nCleaned data with calculated RSI (last 5 rows):")
    display(df_cleaned.tail())

### 4. Interactive Visualization

We now create the plot. The `create_rangebreaks_for_epic` function will automatically generate the correct axis breaks (or none for crypto), ensuring an accurate visual representation.

In [ ]:
if "df_cleaned" in locals() and not df_cleaned.empty:
    rangebreaks = []
    try:
        with get_client(demo_mode=True) as client:
            rangebreaks = create_rangebreaks_for_epic(client, EPIC_TO_VALIDATE)
    except Exception as e:
        logger.error(f"Failed to create rangebreaks, chart may have gaps. Error: {e}")

    fig = make_subplots(
        rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05, row_heights=[0.7, 0.3]
    )

    fig.add_trace(
        go.Candlestick(
            x=df_cleaned.index,
            open=df_cleaned["open"],
            high=df_cleaned["high"],
            low=df_cleaned["low"],
            close=df_cleaned["close"],
            name="Price",
        ),
        row=1,
        col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=df_cleaned.index,
            y=df_cleaned["rsi"],
            mode="lines",
            name=f"RSI({RSI_PERIOD})",
            line=dict(color="purple"),
        ),
        row=2,
        col=1,
    )

    fig.add_hline(
        y=70,
        line_dash="dash",
        line_color="red",
        row=2,
        col=1,
        annotation_text="Overbought",
    )
    fig.add_hline(
        y=30,
        line_dash="dash",
        line_color="green",
        row=2,
        col=1,
        annotation_text="Oversold",
    )

    fig.update_layout(
        title_text=f"Interactive Validation Chart for {EPIC_TO_VALIDATE} ({RESOLUTION})",
        xaxis_rangeslider_visible=False,
        height=800,
        showlegend=False,
    )

    fig.update_xaxes(rangebreaks=rangebreaks)

    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="RSI Value", range=[0, 100], row=2, col=1)

    fig.show()